# How to train a Classifier for MNIST

## Imports

In [1]:
import os
os.chdir("../")

# importing functions and classes from our framework
from activations import Identity, Sigmoid, Tanh, ReLU, LeakyReLU, Softmax, Activation
from loss import MSE, CrossEntropy
from dataset import Dataset
from optimizer import SGD, Adam
from nn import MLP
from layers import Dense, Layer

# other imports
import time
import numpy as np
import logging
logging.basicConfig(level=logging.DEBUG , format='[%(asctime)s] - [%(levelname)s] - %(message)s')

# import the plotting library
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.plotting import figure 
output_notebook()

# get current time in milliseconds
millis = lambda: int(round(time.time() * 1000))
REFRESH_RATE = 5 * 1000 # refresh plot every ___ milliseconds
lastTimes = {"loss": 0, "accuracy": 0}

Loading BokehJS ...

## Live Plots

### Loss function

In [2]:
plotLoss = figure(tools="crosshair,pan,wheel_zoom,box_zoom,reset,tap,box_select,lasso_select", title= "Loss" , plot_height = 350, plot_width = 800)
xLoss = [[],[]]
yLoss = [[],[]]
plotLoss.multi_line(xs=xLoss,ys=yLoss)
targetLoss = show(plotLoss, notebook_handle = True)

### Accuracy

In [3]:
plotAcc = figure(tools="crosshair,pan,wheel_zoom,box_zoom,reset,tap,box_select,lasso_select", title= "Accuracy" , plot_height = 350, plot_width = 800)
xAcc = [[],[]]
yAcc = [[],[]]
plotAcc.multi_line(xs=xAcc,ys=yAcc)
targetAcc = show(plotAcc, notebook_handle = True)

In [4]:
def plot_callback(plotObj, xValues, yValues, targetObject, newX, newY, plotType, train = True):
    # 1. append new data
    i = 0 if train else 1
    xValues[i].append(newX)
    yValues[i].append(newY)
    
    # only update plot after REFRESH_DATE has passed
    if millis() - lastTimes[plotType] > REFRESH_RATE:
        lastTimes[plotType] = millis()
        
        # 2. draw two line plots
        colors = ["mediumblue", "darkorange"]
        legends = ["train", "test"]
        for i in range(2):
            plotObj.line(x=xValues[i],y=yValues[i], color=colors[i], legend_label=legends[i])
        plotObj.legend.location = "top_left"
    
        # 3. push notebook to its target
        push_notebook(handle = targetObject)
    
train_loss_callback = lambda x, y: plot_callback(plotLoss, xLoss, yLoss, targetLoss, x, y, "loss", train = True)
test_loss_callback = lambda x, y: plot_callback(plotLoss, xLoss, yLoss, targetLoss, x, y, "loss", train = False)
train_acc_callback = lambda x, y: plot_callback(plotAcc, xAcc, yAcc, targetAcc, x, y, "accuracy", train = True)
test_acc_callback = lambda x, y: plot_callback(plotAcc, xAcc, yAcc, targetAcc, x, y, "accuracy", train = False)

## Training a neural network

In [5]:
# define dataset on which we train
dataset = Dataset(name = "mnist", train_size = 60000, test_size = 10000, batch_size = 50)

# create a Multi-Layer-Perceptron object
predictor = MLP()
# set the learning rate and optimizer for training
optimizer = Adam(0.01)
# instantiate the layers of the neural network
predictor.addLayer(Dense(inputDim = 28 * 28, outputDim = 100, activation = ReLU(), optimizer = optimizer))
predictor.addLayer(Dense(inputDim = 100, outputDim = 50, activation = ReLU(), optimizer = optimizer))
predictor.addLayer(Dense(inputDim = 50, outputDim = 10, activation = Softmax(), optimizer = optimizer))

print(predictor)

-------------------- MULTI LAYER PERCEPTRON (MLP) --------------------

HIDDEN LAYERS = 1 
TOTAL PARAMETERS = 84060 

 *** 1. Layer: *** 
------------------------
DENSE 784 -> 100 [ReLU]
------------------------
Total parameters: 78500 
---> WEIGHTS: (100, 784)
---> BIASES: (100,)
------------------------

 *** 2. Layer: *** 
-----------------------
DENSE 100 -> 50 [ReLU]
-----------------------
Total parameters: 5050 
---> WEIGHTS: (50, 100)
---> BIASES: (50,)
-----------------------

 *** 3. Layer: *** 
-------------------------
DENSE 50 -> 10 [Softmax]
-------------------------
Total parameters: 510 
---> WEIGHTS: (10, 50)
---> BIASES: (10,)
-------------------------

----------------------------------------------------------------------



In [6]:
# define callback dictionary, in which loss and accuracy callbacks are stored
my_callbacks = {"train_loss": train_loss_callback, "test_loss": test_loss_callback, "train_accuracy": train_acc_callback, "test_accuracy": test_acc_callback}
# works faster without live plots

In [7]:
predictor.train(dataset,loss = CrossEntropy(), epochs = 10,metrics = ["train_loss", "test_loss", "train_accuracy", "test_accuracy"], tensorboard = False, callbacks = my_callbacks)

[2020-07-05 20:39:21,104] - [DEBUG] -  *** EPOCH 1/10 ***
[2020-07-05 20:39:25,533] - [DEBUG] -  *** EPOCH 2/10 ***
[2020-07-05 20:39:29,980] - [DEBUG] -  *** EPOCH 3/10 ***
[2020-07-05 20:39:34,442] - [DEBUG] -  *** EPOCH 4/10 ***
[2020-07-05 20:39:38,944] - [DEBUG] -  *** EPOCH 5/10 ***
[2020-07-05 20:39:43,505] - [DEBUG] -  *** EPOCH 6/10 ***
[2020-07-05 20:39:48,134] - [DEBUG] -  *** EPOCH 7/10 ***
[2020-07-05 20:39:52,861] - [DEBUG] -  *** EPOCH 8/10 ***
[2020-07-05 20:39:57,519] - [DEBUG] -  *** EPOCH 9/10 ***
[2020-07-05 20:40:02,229] - [DEBUG] -  *** EPOCH 10/10 ***


## Compute overall accuracy

In [8]:
testDataset = Dataset(name = "mnist", train_size = 60000, test_size = 10000, batch_size = 10000)

In [9]:
sample = next(testDataset.batches())

In [10]:
# forward propagate the test data
_ = predictor.feedforward(sample[1][0])

In [11]:
print(f"Accuracy on test set: {predictor.getAccuracy(sample[1][1])} %")

Accuracy on test set: 94.97 %


In [12]:
#predictor.save("mnist_classifier")